<a href="https://colab.research.google.com/github/svermeulen1/GDU-Jam-5/blob/main/Game_of_PIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Game of PIG

In [1]:
import jax.numpy as jnp
from jax import random as jrandom
from jax import nn as jnn
from jax import jit
import random
import time

In [2]:
# turn states
ROLL, END, BUST = 'roll', 'end', 'bust'

# game parameters
N_PLAYERS = 3

### Helper functions

In [3]:
def rollDice(random_key):
  return jrandom.randint(random_key, [1], 1, 7, dtype=jnp.dtype('u1'))

### Game Functions

In [4]:
def dumbAI(roll_num, curr_score, curr_run):
  return END if (roll_num >  1 or curr_score + curr_run + roll_num >= 100) else ROLL

def slightlyLessDumbAI(roll_num, curr_score, curr_run):
  return END if (roll_num >  2 or curr_score + curr_run + roll_num >= 100) else ROLL

def reallyDumbAI(roll_num, curr_score, curr_run):
  return END if curr_score + curr_run >= 100 else ROLL

# the number of players is given by the length of the AI array
def playGame(random_key, AIs, verbose=False):
  num_players = len(AIs)
  player_scores = jnp.zeros(num_players)

  # will exit if a player reaches a score greater than 100
  while True:
    for i in range(num_players):
      if verbose:
        print("Player " + str(i+1) + "'s turn (current score: " + str(player_scores[i]) + ")")
      # reset state
      current_state = ROLL
      player_current_run = 0
      num_rolls_in_run = 0

      while current_state == ROLL:
        random_key, subrandom_key = jrandom.split(random_key)
        roll = int(rollDice(subrandom_key))
          

        # game rules (roll 1 -> bust, otherwise add to current run score)
        if roll == 1:
          current_state = BUST
          if verbose: print("---- busted")
        else:
          num_rolls_in_run = num_rolls_in_run + 1
          player_current_run = player_current_run + roll
          if verbose: print("---- rolled: " + str(roll) + " (current run: " + str(player_current_run) + ")")
        
        if current_state != BUST:
          # player chooses whether to continue or accept current runs score
          current_state = AIs[i](num_rolls_in_run, player_scores[i], player_current_run)
          if current_state == END:
            if verbose: print("---- ended turn")
            player_scores = player_scores.at[i].set(player_scores[i] + player_current_run)
          if current_state == ROLL:
            if verbose: print("---- rolling again")
          if (player_scores[i] >= 100):
            print("---- wins!")
            return player_scores, i

def runNGames(N_runs, random_key, AIs, verbose=False):
  num_wins = [0] * len(AIs)
  print("Running..")
  for i in range(N_runs):
    if verbose: print("GAME # " + str(i+1))
    random_key, subrandom_key = jrandom.split(random_key)
    scores, winner = playGame(subrandom_key, AIs, verbose)
    num_wins[winner] = num_wins[winner] + 1

  print("Results are " + str(num_wins))

In [5]:
# initialize 4 AI players
AIs = [dumbAI] * 4
AIs[0] = slightlyLessDumbAI
AIs[1] = reallyDumbAI

random_key = jrandom.PRNGKey(int(time.time()))
runNGames(1, random_key, AIs, True)

Running..
GAME # 1
Player 1's turn (current score: 0.0)
---- rolled: 6 (current run: 6)
---- rolling again
---- rolled: 5 (current run: 11)
---- rolling again
---- rolled: 5 (current run: 16)
---- ended turn
Player 2's turn (current score: 0.0)
---- rolled: 2 (current run: 2)
---- rolling again
---- rolled: 6 (current run: 8)
---- rolling again
---- rolled: 6 (current run: 14)
---- rolling again
---- rolled: 4 (current run: 18)
---- rolling again
---- rolled: 2 (current run: 20)
---- rolling again
---- rolled: 5 (current run: 25)
---- rolling again
---- busted
Player 3's turn (current score: 0.0)
---- rolled: 6 (current run: 6)
---- rolling again
---- rolled: 6 (current run: 12)
---- ended turn
Player 4's turn (current score: 0.0)
---- busted
Player 1's turn (current score: 16.0)
---- rolled: 4 (current run: 4)
---- rolling again
---- rolled: 3 (current run: 7)
---- rolling again
---- rolled: 6 (current run: 13)
---- ended turn
Player 2's turn (current score: 0.0)
---- rolled: 2 (curre